In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import glob
# from rasterstats import zonal_stats

In [2]:
def city_stats(data):
    # Data I need
    # 1. Majority cluster of each zone
    # 2. Majority cluster of each city (all zones)
    # 3. Percentages of each zone
    # 4. Percentages of each city
    histos = data.iloc[:, 2:]
    majority = histos.idxmax(axis=1)
    maj_count = majority.value_counts()

    perc_list = pd.DataFrame()
    for index, row in histos.iterrows():
        percentage = (row / row.sum()) * 100
        perc_list = pd.concat([perc_list, percentage], axis=1)
    perc_list = perc_list.T

    city_sum = perc_list.sum()
    city_perc = (city_sum / city_sum.sum()) * 100

    return majority, maj_count, perc_list, city_perc

In [3]:
slum_zones = gpd.read_file("data/zonal_statistics/capetown_bisecting_5_histogram.dbf")
print(slum_zones)

   Country   City         Settlement Establishe  Last_Updat Population  \
0    Ghana  Accra            Maamobi       1925  03-05-2017      42000   
1    Ghana  Accra            MABROUK       1918  13-01-2018       8500   
2    Ghana  Accra           MAMPROBI       1809  10-10-2016      50000   
3    Ghana  Accra          ABELENKPE       1940  10-10-2016      15000   
4    Ghana  Accra              ABOFO       1918  09-10-2016      32214   
5    Ghana  Accra       ABOSSEY OKAI       1970  03-10-2016      75000   
6    Ghana  Accra           ACHIMOTA       1970  09-10-2016      24000   
7    Ghana  Accra           ADABRAKA       1839  03-05-2017      56000   
8    Ghana  Accra            ADWENBU       1956  10-10-2016      87500   
9    Ghana  Accra              ALAJO       1879  10-10-2016      60000   
10   Ghana  Accra            Amantra       1827  13-01-2018       6600   
11   Ghana  Accra           ANOMULEY       1943  09-10-2016      18000   
12   Ghana  Accra        ASYLUM DOWN  

ERROR 1: PROJ: proj_create_from_database: Open of /home/mendozaemir/anaconda3/envs/remote-sensing-landuse/share/proj failed


In [3]:
cols = ['Settlement', 'Population', 'kyc_pop', 'HISTO_0', 'HISTO_1', 'HISTO_2', 'HISTO_3', 'HISTO_4', 'HISTO_5']
city_data_list = []
city_dict_list = []

for filepath in glob.iglob("data/zonal_statistics/*.dbf"):
    shapefile = gpd.read_file(filepath)
    data = shapefile.loc[:, shapefile.columns.isin(cols)]

    city_data_list.append(data)
    
    majority, maj_count, perc_list, city_perc = city_stats(data)
    city_dict = {'majority': majority, 
                  'maj_count': maj_count, 
                  'perc_list': perc_list, 
                  'city_perc': city_perc}
    
    city_dict_list.append(city_dict)
print(city_data_list[10])

            Settlement   kyc_pop  HISTO_0  HISTO_1  HISTO_2  HISTO_3  HISTO_4  \
0           Commercial    5782.0      1.0      0.0      0.0      0.0      0.0   
1     Kaberera Village    2465.0      0.0      0.0      0.0      1.0      0.0   
2             Kamwanya   12000.0      6.0      0.0      1.0      6.0      1.0   
3      Kandutu Village    1200.0      2.0      1.0      1.0      7.0      2.0   
4              Kuwinda    7000.0      3.0      0.0      0.0      3.0      1.0   
5             Kibagare   30000.0     25.0      0.0      2.0     17.0      0.0   
6   Githongori Village    6000.0     33.0      6.0     12.0     44.0     17.0   
7          Fuata Nyayo   22388.0      0.0      0.0      0.0      2.0      0.0   
8            Mariguini   30708.0      6.0      0.0      0.0      4.0      0.0   
9           Laini Saba  102240.0     32.0      0.0      1.0     10.0      0.0   
10          Kambi Muru    3591.0     29.0      0.0      0.0     10.0      0.0   
11              Kianda   600

ERROR 1: PROJ: proj_create_from_database: Open of /home/mendozaemir/anaconda3/envs/remote-sensing-landuse/share/proj failed


In [5]:
# 0. accra
# 1. athiriver
# 2. capetown
# 3 daressalaam
# 4. kampala
# 5. kisumu
# 6. lagos
# 7. malabon
# 8. portharcourt
# 9. nairobi east
# 10. nairobi center

for city in city_dict_list:
    maj_indices = [index for (index, item) in enumerate(list(city.values())[0]) if item == 'HISTO_4']
    

# Parses through dictionary list and grabs all indices with HISTO_x as majority
# replace k with numbers above corresponding to domains
k=3
maj_indices = [index for (index, item) in enumerate(list(city_dict_list[k].values())[0]) if item == 'HISTO_0']
print(maj_indices)
# For grabbing population
accrahisto3 = [city_data_list[k].loc[i] for i in maj_indices]
popsum = 0
for city in accrahisto3:
    #some files use population some use kyc_pop
    popsum = popsum + int(city['Population'])
print(popsum)

print(list(city_dict_list[k].values())[3])

[5, 6, 8, 9, 10, 12, 13, 15, 16, 17, 19, 31, 34, 35, 36, 37, 39, 41, 42, 47, 49, 53, 54, 58, 62, 63, 64, 66, 68, 70, 75, 77]
1030485
HISTO_0    23.505535
HISTO_1     4.716648
HISTO_2    11.747007
HISTO_3    16.295381
HISTO_4    34.348074
HISTO_5     9.387356
dtype: float64


In [32]:
city_data_list[0]['Population']

0      42000
1       8500
2      50000
3      15000
4      32214
5      75000
6      24000
7      56000
8      87500
9      60000
10      6600
11     18000
12     25000
13    145000
14      7200
15      4000
16     16980
17    120000
18     25000
19     60000
20     96000
21    125000
22    250000
23      7000
24    144000
25     12000
26      7500
27     60000
28     25600
29       900
30      2000
31      7500
32      9600
33    141400
34     15000
Name: Population, dtype: object